In [23]:
import numpy as np
 
import numba
from numba import cuda
from scipy.stats import pearsonr
from time import time
import math  
 

In [106]:
# playing around with numba for the first time

an_array = np.random.random((15000,15000))

@cuda.jit
def pearson(an_array, out):
    x= cuda.threadIdx.x # this row
    M = 13000
    if (x<M):
        data1 = an_array[x]
        for index in range(x, 15000): # against every other row
            data2 = an_array[index] 
             
            sum1 = 0.
            sum2 = 0.
            for i in range(M):
                sum1 += data1[i]
                sum2 += data2[i]
            mean1 = sum1 / M
            mean2 = sum2 / M

            var_sum1 = 0.
            var_sum2 = 0.
            cross_sum = 0.
            for i in range(M):
                var_sum1 += (data1[i] - mean1) ** 2
                var_sum2 += (data2[i] - mean2) ** 2
                cross_sum += (data1[i] * data2[i])

            std1 = (var_sum1 / M) ** .5
            std2 = (var_sum2 / M) ** .5
            cross_mean = cross_sum / M

            out[x,index] = (cross_mean - mean1 * mean2) / (std1 * std2)

    
out=np.zeros((15000,15000)) 
start = time()
pearson[1,100](an_array, out) 
print(time() - start)

64.56297898292542
